<a href="https://colab.research.google.com/github/praffuln/langchain/blob/master/langchain-2/langchain_9_youtubeScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install langchain
!pip install openAI
!pip install wikipedia
!pip install huggingface_hub
!pip install InstructorEmbedding
!pip install google-search-results
!pip install unstructured
!pip install libmagic
!pip install python-magic
!pip install python-magic-bin
#Install faiss Packages
!pip install faiss-cpu
!pip install sentence-transformers
!pip install wolframalpha

!pip install youtube-transcript-api
!pip install pytube
!pip install python-dotenv

ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


# API

In [2]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""


# *1. Simple Videos*

In [8]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=True)


In [9]:
result = loader.load()


In [10]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)


<class 'list'>
Found video from Saturday Night Live that is 224 seconds long

[Document(page_content='LADIES AND GENTLEMEN, PEDRO PASCAL! [ CHEERS AND APPLAUSE ] >> THANK YOU, THANK YOU. THANK YOU VERY MUCH. I\'M SO EXCITED TO BE HERE. THANK YOU. I SPENT THE LAST YEAR SHOOTING A SHOW CALLED "THE LAST OF US" ON HBO. FOR SOME HBO SHOES, YOU GET TO SHOOT IN A FIVE STAR ITALIAN RESORT SURROUNDED BY BEAUTIFUL PEOPLE, BUT I SAID, NO, THAT\'S TOO EASY. I WANT TO SHOOT IN A FREEZING CANADIAN FOREST WHILE BEING CHASED AROUND BY A GUY WHOSE HEAD LOOKS LIKE A GENITAL WART. IT IS AN HONOR BEING A PART OF THESE HUGE FRANCHISEs LIKE "GAME OF THRONES" AND "STAR WARS," BUT I\'M STILL GETTING USED TO PEOPLE RECOGNIZING ME. THE OTHER DAY, A GUY STOPPED ME ON THE STREET AND SAYS, MY SON LOVES "THE MANDALORIAN" AND THE NEXT THING I KNOW, I\'M FACE TIMING WITH A 6-YEAR-OLD WHO HAS NO IDEA WHO I AM BECAUSE MY CHARACTER WEARS A MASK THE ENTIRE SHOW. THE GUY IS LIKE, DO THE MANDO VOICE, BUT IT\'S LIKE A BEDRO

In [11]:
from langchain.document_loaders import YoutubeLoader
from langchain import HuggingFaceHub
from langchain.chains.summarize import load_summarize_chain


In [12]:
# Load up your LLM
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature": 0.5}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [13]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)


'Pedro Pascal, who plays Yoda in "Star Wars: The Force Awakens," discusses his family, his acting career and his love for the Star Wars franchise.'

# *2. Long Videos*

In [14]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=6Ub7Z1AGIuk", add_video_info=True)


In [15]:
result = loader.load()


In [16]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)


ValueError: ignored

Problem, your transcript/document is too long



In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)


In [23]:
texts = text_splitter.split_documents(result)


In [24]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"somehow five years later covet hits and you know what they decide to do they're like you know what let's go all in on creating like safety safety stuff for covert like masks respirators um gloves things like that and they become one of the like core providers right at the start of covet they shifted all their production manufacturing stuff they're like forget the vodka bottles forget the LED screens we need n95 masks and we need respirators and we need you know these testing kits and we need and they started producing this stuff and they started just winning all these contracts [Music] we're live Sean uh we have a bunch of stuff you you have a you have a a full menu over here and so do I yeah all right I want you to look at my menu here and I want you to just pick something off the menu go ahead and I'll Riff Off it we don't prefer p

'Jimmy Fallon is a skeptic about the success of his neighbor, a wealthy man. Fallon: "I\'m you know I get out there and I collect signals is this you know where where do I place this person on the uh the Billy scale like are they you know are they on their way up are they are they have they made it in life are they really balling out of control where where is this person these people have been balling out of control like when Christmas time came up there was just like the wife went outside and it was just pointing at windows and all of a sudden the house was lit up like a Christmas tree they had like so they had almost like decked out things and they go hey hey um invite your kids over uh this Thursday we have a snow machine they\'re we\'re building a "snow" man." On a Thursday afternoon, a truck came to the house and made a snow hill.'

# 3. Multiple Videos

In [25]:
youtube_url_list = ["https://www.youtube.com/watch?v=AXq0QHUwmh8", "https://www.youtube.com/watch?v=EwHrjZxAT7g"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()

    texts.extend(text_splitter.split_documents(result))


In [26]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)


'The Sputter and Golf, you can build your own backyard golf course. The cost of a golf green is based on the area, the type of grass, and the quality of the soil. The cost of the golf green was $10,500. The cost of the fairway was $3,800. The cost of the irrigation system is $3,800. In this episode of Spud Run Golf, we talk about the cost of building a golf course, the spuds, and what\'s next. From the "Game of Thrones" set to The Last of Us, Bella Ramsey and Pedro Pascal share their thoughts on the experience. "What was your first job?" Game of Thrones. PEDRO and BELLA are telling each other their childhood jobs. PEDRO: At 11 years old, I was a bar maid. BELLA: At 11 years old, I got paid like a tiny amount of money for singing in a cathedral choir. There you go. Mine was, I don\'t know, a coffee-- Like a barista. I\'ve always wanted to be a barista. PEDRO: At a coffee shop in Manhattan. Can you do the... (MIMICS CAPPUCCINO MACHINE) Yeah, back then. This was-- -So you couldn\'t now. Y